# init

In [9]:
import string
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", None)

def drop_all_none(df):
    return (
        df
        .dropna(how="all", axis=0)  # 行方向：全None行を削除
        .dropna(how="all", axis=1)  # 列方向：全None列を削除
    )

# examples

In [10]:
df1 = pd.DataFrame(
    [
        [None, None, None, None, None],      # 行0：全None
        [1.0,  "a",  None,  2.0,  "x"],      # 行1
        [None, None, None, None, None],      # 行2：全None
        [3.0,  "b",  None,  None, "y"],      # 行3
        [5.0,  "c",  None,  7.0,  "z"],      # 行4
    ],
    columns=["A", "B", "C", "D", "E"]
)


df2 = pd.DataFrame(
    [
        [1.5,  "a",  2.5,  None, "x"],       # 行0
        [None, None, None, None, None],      # 行1：全None
        [2.0,  "b",  3.0,  None, "y"],       # 行2
        [None, None, None, None, None],      # 行3：全None
        [5.5,  "c",  6.5,  None, "z"],       # 行4
    ],
    columns=["A", "B", "C", "D", "E"]
)


# process

### Fail in concat 1

In [11]:
df1 = drop_all_none(df1)
df2 = drop_all_none(df2)

print(df1)
print(df2)

df_concat = pd.concat([df1, df2], axis=0)
print(df_concat)

     A  B    D  E
1  1.0  a  2.0  x
3  3.0  b  NaN  y
4  5.0  c  7.0  z
     A  B    C  E
0  1.5  a  2.5  x
2  2.0  b  3.0  y
4  5.5  c  6.5  z
     A  B    D  E    C
1  1.0  a  2.0  x  NaN
3  3.0  b  NaN  y  NaN
4  5.0  c  7.0  z  NaN
0  1.5  a  NaN  x  2.5
2  2.0  b  NaN  y  3.0
4  5.5  c  NaN  z  6.5


### Fail in concat 2

In [12]:
common_cols = df1.columns.union(df2.columns)
df_concat = pd.concat([
    df1.reindex(columns=common_cols),
    df2.reindex(columns=common_cols),
    ], axis=0, ignore_index=True,)

print(df_concat)

     A  B    C    D  E
0  1.0  a  NaN  2.0  x
1  3.0  b  NaN  NaN  y
2  5.0  c  NaN  7.0  z
3  1.5  a  2.5  NaN  x
4  2.0  b  3.0  NaN  y
5  5.5  c  6.5  NaN  z


### Success in concat df body

In [13]:
def df_concat_body(df1, df2):
    orig_cols_df1 = df1.columns.copy()

    df1_reset = df1.copy()
    df2_reset = df2.copy()

    df1_reset.columns = range(df1_reset.shape[1])
    df2_reset.columns = range(df2_reset.shape[1])

    df_concat = pd.concat(
        [df1_reset, df2_reset],
        axis=0,
        ignore_index=True
    )
    print(df_concat)
    df_concat.columns=orig_cols_df1
    return df_concat

df_concat = df_concat_body(df1, df2) # df1=df_left, df2=df_right
print(df_concat)

n_cols = df_concat.shape[1]
labels = list(string.ascii_lowercase[:n_cols])
df_concat.columns = labels
print(df_concat)

     0  1    2  3
0  1.0  a  2.0  x
1  3.0  b  NaN  y
2  5.0  c  7.0  z
3  1.5  a  2.5  x
4  2.0  b  3.0  y
5  5.5  c  6.5  z
     A  B    D  E
0  1.0  a  2.0  x
1  3.0  b  NaN  y
2  5.0  c  7.0  z
3  1.5  a  2.5  x
4  2.0  b  3.0  y
5  5.5  c  6.5  z
     a  b    c  d
0  1.0  a  2.0  x
1  3.0  b  NaN  y
2  5.0  c  7.0  z
3  1.5  a  2.5  x
4  2.0  b  3.0  y
5  5.5  c  6.5  z


# unpivot

===== Excelから読み込む想定 =====

In [ ]:
# skiprowsで102行スキップして、103行目を列名として読み込む
# df = pd.read_excel("sample.xlsx", header=102)

# 疑似データフレーム（103行目以降のデータ相当）
columns = list("DEFGHIJKLMNOPQ")  # D〜Q
data = [
    ["数学", 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    ["英語", 20, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34],
    ["理科", 30, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44],
]
df = pd.DataFrame(data, columns=columns)
print("=== 読み込み直後の df ===")
print(df)

# D列以降だけを抽出（例：A,B,C は無視）
df = df.loc[:, "D":]

# ===== 列名の設定 =====
# 現在の列リストを確認
cols = df.columns.tolist()

# D列だけ "科目" に
cols[0] = "科目"

# E〜Q列は "列E", "列F", ... にリネーム
for i, c in enumerate(cols[1:], start=1):
    # E列以降はアルファベットを使って命名
    cols[i] = f"列{chr(ord('E') + i - 1)}"
df.columns = cols

df = df.drop(columns=["列E"])  # E列だけ削除

print("=== 列名設定後 ===")
print(df.head())

# 列N, O, P は "列N", "列O", "列P" に対応している
df["列L'"] = df[["列N", "列O", "列P"]].sum(axis=1)


# 対象列を指定
pivot_cols = [ "列F", "列G", "列H", "列I", "列J", "列K", "列L", "列L'"]

# ピボット解除
df_unpivot = df.melt(
    id_vars=[col for col in df.columns if col not in pivot_cols],  # それ以外の列は保持
    value_vars=pivot_cols,
    var_name="列名",
    value_name="値"
)
df = df_unpivot[["科目", "列名",  "値", "列M", "列N", "列O", "列P", "列Q"]]
df = df.sort_values(by=["科目", "列名"], ascending=True)   # 昇順
print("=== ピボット解除後 ===")
print(df.head(20))
